In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', 50)
from datetime import datetime

In [2]:
df = pd.read_csv('Data Set 2.csv')
df.head()

,OrderId,CustomerId,LocationId,channel,Amount,OrderDate,BillNo
0,35623100,6662,4416,foodpanda,267,1/1/2018,DEL
1,35622814,8237,4002,zomato,1051,1/1/2018,DEL
2,35623284,11912,4416,takeaway,714,1/1/2018,DEL
3,35631171,14563,218,zomato,834,1/1/2018,GGN
4,35635515,14833,295,call,544,1/1/2018,DEL


In [3]:
df = df[df['BillNo'] == 'NOI']

In [4]:
df['OrderDate']= pd.to_datetime(df['OrderDate'])

In [5]:
df['OrderPeriod'] = df.OrderDate.apply(lambda x: x.strftime('%Y-%m'))

In [6]:
df.head()

,OrderId,CustomerId,LocationId,channel,Amount,OrderDate,BillNo,OrderPeriod
7,35627358,22620,4003,zomato,320,2018-01-01,NOI,2018-01
20,35627040,75433,4003,foodpanda,294,2018-01-01,NOI,2018-01
49,35627299,138913,4003,zomato,869,2018-01-01,NOI,2018-01
53,35626991,145362,4003,foodpanda,372,2018-01-01,NOI,2018-01
61,35627721,158035,4003,call,2961,2018-01-01,NOI,2018-01


In [7]:
#Revenue
RevByMonth = df.groupby('OrderPeriod')['Amount'].sum() 
RevByMonth = pd.DataFrame(RevByMonth)
RevByMonth.head()

,Amount
OrderPeriod,
2018-01,1512184
2018-02,2050810
2018-03,1383136
2018-04,1386582
2018-05,1019109


In [8]:
#Orders
OrderByMonth = df.groupby('OrderPeriod')['OrderId'].count() #Orders
OrderByMonth = pd.DataFrame(OrderByMonth)
OrderByMonth.head()

,OrderId
OrderPeriod,
2018-01,2537
2018-02,2480
2018-03,2478
2018-04,2394
2018-05,1802


In [9]:
#Customers
CusByMonth = df.groupby("OrderPeriod").agg({"CustomerId": pd.Series.nunique}) 
CusByMonth.head()

,CustomerId
OrderPeriod,
2018-01,2036
2018-02,1992
2018-03,1970
2018-04,1851
2018-05,1463


In [10]:
#Repeat Customers
grouped = df.groupby(['OrderPeriod', 'CustomerId'])
RC = grouped.agg({'OrderId': pd.Series.nunique})
RC.head()

OrderId
OrderPeriod CustomerId         
2018-01     5838              1
            8616              2
            11164             1
            14069             1
            16270             1

In [11]:
RCNew = RC[RC.OrderId > 1]
RCNew.head()

OrderId
OrderPeriod CustomerId         
2018-01     8616              2
            18067             3
            20650             2
            35925             4
            75433             2

In [12]:
RepCustomers = RCNew.groupby('OrderPeriod').count() 
RepCustomers.rename(columns={'OrderId': 'RepeatCustomers'}, inplace=True)
RepCustomers.head()

,RepeatCustomers
OrderPeriod,
2018-01,318
2018-02,314
2018-03,322
2018-04,290
2018-05,216


In [13]:
#New Customers
df.set_index('CustomerId', inplace=True)
df['CohortGroup'] = df.groupby(level=0)['OrderDate'].min().apply(lambda x: x.strftime('%Y-%m'))
df.reset_index(inplace=True)
df.head()

,CustomerId,OrderId,LocationId,channel,Amount,OrderDate,BillNo,OrderPeriod,CohortGroup
0,22620,35627358,4003,zomato,320,2018-01-01,NOI,2018-01,2018-01
1,75433,35627040,4003,foodpanda,294,2018-01-01,NOI,2018-01,2018-01
2,138913,35627299,4003,zomato,869,2018-01-01,NOI,2018-01,2018-01
3,145362,35626991,4003,foodpanda,372,2018-01-01,NOI,2018-01,2018-01
4,158035,35627721,4003,call,2961,2018-01-01,NOI,2018-01,2018-01


In [14]:
grouped = df.groupby(['CohortGroup', 'OrderPeriod'])
newC = grouped.agg({'CustomerId': pd.Series.nunique,
                       'OrderId': pd.Series.nunique})
newC.head()

CustomerId  OrderId
CohortGroup OrderPeriod                     
2018-01     2018-01            2036     2537
            2018-02             427      721
            2018-03             382      628
            2018-04             320      516
            2018-05             231      327

In [15]:
def cohort_period(df):
    
    df['CohortPeriod'] = np.arange(len(df)) + 1
    return df

newC = newC.groupby(level=0).apply(cohort_period)
newC.head()

CustomerId  OrderId  CohortPeriod
CohortGroup OrderPeriod                                   
2018-01     2018-01            2036     2537             1
            2018-02             427      721             2
            2018-03             382      628             3
            2018-04             320      516             4
            2018-05             231      327             5

In [16]:
NewCustomers = newC[newC['CohortPeriod'] == 1]

In [17]:
NC = NewCustomers.reset_index()

In [18]:
NC = NC.drop(['CohortGroup'], axis =1)

In [19]:
NC = NC.set_index('OrderPeriod')

In [20]:
NC = NC[['CustomerId']]

In [21]:
NC.rename(columns={'CustomerId': 'NewCustomers'}, inplace=True)

In [22]:
NC.head()

,NewCustomers
OrderPeriod,
2018-01,2036
2018-02,1565
2018-03,1357
2018-04,1184
2018-05,923


In [23]:
dfMerge = pd.concat([RevByMonth, OrderByMonth, CusByMonth, NC,RepCustomers],axis=1,sort=False).reset_index()
dfMerge.rename(columns = {'index':'OrderPeriod'})
dfMerge.rename(columns={'Amount': 'Revenue',
                        'OrderId': 'Orders',
                        'CustomerId':'Customers'}, inplace=True)

In [24]:
dfMerge['Frequency'] = dfMerge['Orders']/dfMerge['Customers']

In [25]:
dfMerge.head()

,OrderPeriod,Revenue,Orders,Customers,NewCustomers,RepeatCustomers,Frequency
0,2018-01,1512184,2537,2036,2036,318,1.246071
1,2018-02,2050810,2480,1992,1565,314,1.244980
2,2018-03,1383136,2478,1970,1357,322,1.257868
3,2018-04,1386582,2394,1851,1184,290,1.293355
4,2018-05,1019109,1802,1463,923,216,1.231716


In [26]:
dfMerge

,OrderPeriod,Revenue,Orders,Customers,NewCustomers,RepeatCustomers,Frequency
0,2018-01,1512184,2537,2036,2036,318,1.246071
1,2018-02,2050810,2480,1992,1565,314,1.244980
2,2018-03,1383136,2478,1970,1357,322,1.257868
3,2018-04,1386582,2394,1851,1184,290,1.293355
4,2018-05,1019109,1802,1463,923,216,1.231716
5,2018-06,994666,1919,1695,1289,155,1.132153
6,2018-07,2325545,3954,3357,2660,420,1.177837
7,2018-08,1675902,2941,2392,1613,338,1.229515
8,2018-09,2044779,3974,3169,2334,484,1.254023
9,2018-10,3843890,5495,4136,3203,718,1.328578


In [27]:
dfMerge = dfMerge.set_index('OrderPeriod')

In [28]:
dfMerge.transpose()

OrderPeriod,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04
Revenue,1.512184e+06,2.050810e+06,1.383136e+06,1.386582e+06,1.019109e+06,994666.000000,2.325545e+06,1.675902e+06,2.044779e+06,3.843890e+06,5.236253e+06,4.699856e+06,3.132024e+06,3.242793e+06,4.355617e+06,1.311443e+06
Orders,2.537000e+03,2.480000e+03,2.478000e+03,2.394000e+03,1.802000e+03,1919.000000,3.954000e+03,2.941000e+03,3.974000e+03,5.495000e+03,8.049000e+03,7.186000e+03,4.938000e+03,6.899000e+03,1.062900e+04,3.285000e+03
Customers,2.036000e+03,1.992000e+03,1.970000e+03,1.851000e+03,1.463000e+03,1695.000000,3.357000e+03,2.392000e+03,3.169000e+03,4.136000e+03,5.986000e+03,5.568000e+03,3.934000e+03,5.498000e+03,7.959000e+03,2.785000e+03
NewCustomers,2.036000e+03,1.565000e+03,1.357000e+03,1.184000e+03,9.230000e+02,1289.000000,2.660000e+03,1.613000e+03,2.334000e+03,3.203000e+03,4.480000e+03,3.821000e+03,2.488000e+03,3.570000e+03,5.366000e+03,1.546000e+03
RepeatCustomers,3.180000e+02,3.140000e+02,3.220000e+02,2.900000e+02,2.160000e+02,155.000000,4.200000e+02,3.380000e+02,4.840000e+02,7.180000e+02,1.236000e+03,9.740000e+02,6.130000e+02,8.940000e+02,1.565000e+03,3.640000e+02
Frequency,1.246071e+00,1.244980e+00,1.257868e+00,1.293355e+00,1.231716e+00,1.132153,1.177837e+00,1.229515e+00,1.254023e+00,1.328578e+00,1.344637e+00,1.290589e+00,1.255211e+00,1.254820e+00,1.335469e+00,1.179533e+00


In [29]:
dfMerge.to_excel(r'C:\Users\user\Desktop\Xeno\NOI Report.xlsx')